# Iris Dataset (built-in dataset)
- The dataset contains a set of 150 records under five attributes sepal
length, sepal width, petal length, petal width and species.

- This data sets consists of 3 different types of irises’ ( Setosa , Versicolour , and Virginica) petal and sepal length, stored in a 150x4 numpy.ndarray.

- The rows being the samples and the columns being: Sepal Length, Sepal Width,
Petal Length and Petal Width.

# 0.Authors

### Sohaila Diab    
### Osama Ibrahim    
### Rawan Khaled    


# 1. Imports

In [ ]:
import sklearn as sk
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import seaborn as sns
sns.set( color_codes=True)
# Evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from plotly.express import scatter_3d

# Modeling 
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import BernoulliNB


#decision boundery
from mlxtend.plotting import plot_decision_regions


# 2. Load Data

In [ ]:
# Function to convert sklearn dataset to dataFrame

def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['species'] = pd.Series(sklearn_dataset.target) # Changed col name from 'target' to 'species'
    return df

In [ ]:
#load data from sklearn
iris = sklearn_to_df(sk.datasets.load_iris())
iris.head()

In [ ]:
iris.shape

# 3.Understanding the Data

In [ ]:
iris.info()

In [ ]:
iris.describe()

In [ ]:
#Target Distribution
iris['species'].value_counts().plot(kind='bar')
plt.title('Target Distribution')
plt.xlabel('species')
plt.ylabel('Count')

In [ ]:
# sepal length (cm) Distribution
iris['sepal length (cm)'].plot(kind='hist')
plt.title('sepal length (cm) Distribution')
plt.xlabel('sepal length (cm)')
plt.ylabel('Frequency')

In [ ]:
#sepal width (cm) Distribution
iris['sepal width (cm)'].plot(kind='hist')
plt.title('sepal width (cm) Distribution')
plt.xlabel('sepal width (cm)')
plt.ylabel('Frequency')


In [ ]:
iris.plot(kind="scatter", x="sepal length (cm)", y="sepal width (cm)")

In [ ]:
sns.FacetGrid(iris, hue="species", size=5) \
   .map(plt.scatter, "sepal length (cm)", "sepal width (cm)") \
   .add_legend()


In [ ]:
# petal length (cm) Distribution
iris['petal length (cm)'].plot(kind='hist')
plt.title('petal length (cm) Distribution')
plt.xlabel('petal length (cm)')
plt.ylabel('Frequency')


In [ ]:
#petal width (cm) Distribution
iris['petal width (cm)'].plot(kind='hist')
plt.title('petal width (cm) Distribution')
plt.xlabel('petal width (cm)')
plt.ylabel('Frequency')

In [ ]:
iris.plot(kind="scatter", x="petal length (cm)", y="petal width (cm)")

In [ ]:
sns.FacetGrid(iris, hue="species", size=5) \
   .map(plt.scatter, "petal length (cm)", "petal width (cm)") \
   .add_legend()


In [ ]:
ax = sns.boxplot(x="species", y="petal length (cm)", data=iris)
ax = sns.stripplot(x="species", y="petal length (cm)", data=iris, jitter=True, edgecolor="gray")


# 4. Split Data

In [ ]:
# Function to split train & test & validate
def train_validate_test_split(data, labels, testRatio=0.3, valRatio=0.3):
  """
  Inputs:
  data(numpy array)
  labels (numpy array)
  testRatio (Float)
  valRatio (Float)
  Outputs:
  X_train (numpy array)
  X_val (numpy array)
  X_test (numpy array)
  y_train (numpy array)
  y_val (numpy array)
  y_test (numpy array)

  
  """
  # Data size
  n = len(data)
  val_n = int(valRatio*n)
  test_n = int(testRatio*n)
  train_n = n-val_n-test_n # do not do `int((1-valRatio-testRatio)*n)` since rows might go missing when converting to int

  # # Shuffle indices and data
  np.random.seed(5) # set seed to keep randomness constant each run
  idx = np.arange(n)
  np.random.shuffle(idx)

  X, y = data[idx], labels[idx]

  # Split the data
  # Train
  X_train, y_train = X[:train_n], y[:train_n]
  # Validation
  X_val, y_val = X[train_n : train_n+val_n], y[train_n : train_n+val_n]
  # Test
  X_test, y_test = X[train_n+val_n:], y[train_n+val_n:]

  return X_train, X_val, X_test, y_train, y_val, y_test

In [ ]:
X = iris.iloc[:,:-1].to_numpy()
y = iris.species.to_numpy()

X_train, X_val, X_test, y_train, y_val, y_test = train_validate_test_split(X,y,0.3,0.3)

print(f'Train X: {len(X_train)}, Train y: {len(y_train)}')
print(f'Val X: {len(X_val)}, Val y: {len(y_val)}')
print(f'Test X: {len(X_test)}, Test y: {len(y_test)}')

print(f'Total: {len(X_train)+len(X_val)+len(X_test)}')

## Accuracy Function

In [ ]:
# Function to calculate accuracy
def calculate_accuracy(y_actual, y_pred):
  return sum(y_actual == y_pred)/len(y_actual) 

# 5. Train - Bayes Classifier


## 5.1. Gaussian Naive Bayes

**Gaussian Naive Bayes**

(Gaussian) Naive Bayes assumes that each class follow a Gaussian distribution.

## <center> $P(x_i \mid y) = \frac{1}{\sqrt{2\pi\sigma^2_y}} \exp\left(-\frac{(x_i - \mu_y)^2}{2\sigma^2_y}\right)$

In [ ]:
# Model Training
GNBclf = GaussianNB()
GNBmodel = GNBclf.fit(X_train, y_train)
# Model Prediction
GNB_pred = GNBclf.predict(X_val)

# Model Accuracy
GNB_acc = calculate_accuracy(y_val, GNB_pred)
print('Gaussian Naive Bayes accuracy score: {0:0.4f}'. format(GNB_acc))
print(classification_report(y_val, GNB_pred))

In [ ]:
# Creating  a confusion matrix,which compares the y_test and y_pred
cm = confusion_matrix(y_val, GNB_pred)
# Creating a dataframe for a array-formatted Confusion matrix,so it will be easy for plotting.
cm_df = pd.DataFrame(cm,
                     index = ['Setosa (0)','Versicolour (1)','Virginica (2)'], 
                     columns = ['Setosa (0)','Versicolour (1)','Virginica (2)'])
#Plotting the confusion matrix
plt.figure(figsize=(8,5))
sns.heatmap(cm_df, annot=True)   
plt.title('Confusion Matrix')
plt.ylabel('ACTUAL VALUES')
plt.xlabel('PREDICTED VALUES')
plt.show()

## 5.2. Multinomial Naive Bayes

implements the naive Bayes algorithm for multinomially distributed data, and is one of the two classic naive Bayes variants used in text classification (where the data are typically represented as word vector counts, although tf-idf vectors are also known to work well in practice). The distribution is parametrized by vectors  for each class , where  is the number of features (in text classification, the size of the vocabulary) and  is the probability  of feature  appearing in a sample belonging to class .

The parameters  is estimated by a smoothed version of maximum likelihood, i.e. relative frequency counting:
## <center> $\hat{\theta}_{yi} = \frac{ N_{yi} + \alpha}{N_y + \alpha n}$

In [ ]:
# Model Training
Multiclf = MultinomialNB()
Multimodel = Multiclf.fit(X_train, y_train)
# Model Prediction
Multi_pred = Multimodel.predict(X_val)
# Model Accuracy
Multi_acc = calculate_accuracy(y_val, Multi_pred)
print('Multinomial Naive Bayes accuracy score: {0:0.4f}'.format(Multi_acc))
print(classification_report(y_val, Multi_pred))

In [ ]:
# Creating  a confusion matrix,which compares the y_test and y_pred
cm = confusion_matrix(y_val, Multi_pred)
# Creating a dataframe for a array-formatted Confusion matrix,so it will be easy for plotting.
cm_df = pd.DataFrame(cm,
                     index = ['Setosa (0)','Versicolour (1)','Virginica (2)'], 
                     columns = ['Setosa (0)','Versicolour (1)','Virginica (2)'])
#Plotting the confusion matrix
plt.figure(figsize=(8,5))
sns.heatmap(cm_df, annot=True)   
plt.title('Confusion Matrix')
plt.ylabel('ACTUAL VALUES')
plt.xlabel('PREDICTED VALUES')
plt.show()

## 5.3. Complement Naive Bayes

 implements the complement naive Bayes (CNB) algorithm. CNB is an adaptation of the standard multinomial naive Bayes (MNB) algorithm that is particularly suited for imbalanced data sets. Specifically, CNB uses statistics from the complement of each class to compute the model’s weights. The inventors of CNB show empirically that the parameter estimates for CNB are more stable than those for MNB. Further, CNB regularly outperforms MNB (often by a considerable margin) on text classification tasks. The procedure for calculating the weights is as follows:
 ## <center> $\begin{align}\begin{aligned}\hat{\theta}_{ci} = \frac{\alpha_i + \sum_{j:y_j \neq c} d_{ij}} {\alpha + \sum_{j:y_j \neq c} \sum_{k} d_{kj}}\\w_{ci} = \log \hat{\theta}_{ci}\\w_{ci} = \frac{w_{ci}}{\sum_{j} |w_{cj}|}\end{aligned}\end{align}$

In [ ]:
# Model Training
Compclf = ComplementNB()
Compmodel = Compclf.fit(X_train, y_train)
# Model Prediction
Comp_pred = Compmodel.predict(X_val)
# Model Accuracy
Comp_acc = calculate_accuracy(y_val, Comp_pred)
print('Complement Naive Bayes accuracy score: {0:0.4f}'.format(Comp_acc))
print(classification_report(y_val, Comp_pred))

In [ ]:
# Creating  a confusion matrix,which compares the y_test and y_pred
cm = confusion_matrix(y_val, Comp_pred)
# Creating a dataframe for a array-formatted Confusion matrix,so it will be easy for plotting.
cm_df = pd.DataFrame(cm,
                     index = ['Setosa (0)','Versicolour (1)','Virginica (2)'], 
                     columns = ['Setosa (0)','Versicolour (1)','Virginica (2)'])
#Plotting the confusion matrix
plt.figure(figsize=(8,5))
sns.heatmap(cm_df, annot=True)   
plt.title('Confusion Matrix')
plt.ylabel('ACTUAL VALUES')
plt.xlabel('PREDICTED VALUES')
plt.show()

## 5.4. Bernoulli Naive Bayes

implements the naive Bayes training and classification algorithms for data that is distributed according to multivariate Bernoulli distributions; i.e., there may be multiple features but each one is assumed to be a binary-valued (Bernoulli, boolean) variable. Therefore, this class requires samples to be represented as binary-valued feature vectors; if handed any other kind of data, a BernoulliNB instance may binarize its input (depending on the binarize parameter).

The decision rule for Bernoulli naive Bayes is based on
## <center> $P(x_i \mid y) = P(x_i = 1 \mid y) x_i + (1 - P(x_i = 1 \mid y)) (1 - x_i)$

In [ ]:
# Model Training
Bernclf = BernoulliNB()
Bernmodel = Bernclf.fit(X_train, y_train)
# Model Prediction
Bern_pred = Bernmodel.predict(X_val)
# Model Accuracy
Bern_acc = calculate_accuracy(y_val, Bern_pred)
print('Bernoulli Naive Bayes accuracy score: {0:0.4f}'.format(Bern_acc))
print(classification_report(y_val, Bern_pred))

In [ ]:
# Creating  a confusion matrix,which compares the y_test and y_pred
cm = confusion_matrix(y_val, Bern_pred)
# Creating a dataframe for a array-formatted Confusion matrix,so it will be easy for plotting.
cm_df = pd.DataFrame(cm,
                     index = ['Setosa (0)','Versicolour (1)','Virginica (2)'], 
                     columns = ['Setosa (0)','Versicolour (1)','Virginica (2)'])
#Plotting the confusion matrix
plt.figure(figsize=(8,5))
sns.heatmap(cm_df, annot=True)   
plt.title('Confusion Matrix')
plt.ylabel('ACTUAL VALUES')
plt.xlabel('PREDICTED VALUES')
plt.show()

# 6. Decision Boundary

## 2-D

In [ ]:
#Draw decision boundaries
X_decision = X_train[:, [0, 2]]


# Training a classifier
GNBclf_D = GaussianNB()
GNBmodel_D = GNBclf_D.fit(X_decision, y_train)
GNBmodel_D

# Plotting decision regions
plot_decision_regions(X_decision, y_train, clf=GNBclf_D, legend=2)

# Adding axes annotations
plt.xlabel('sepal length [cm]')
plt.ylabel('petal length [cm]')
plt.title('GNBclf on Iris')
plt.show()


In [ ]:
#Draw decision boundaries
X_decision = X_train[:, [0, 2]]


# Training a classifier
Multiclf_D =  MultinomialNB()
Multimodel_D = Multiclf_D.fit(X_decision, y_train)
Multimodel_D

# Plotting decision regions
plot_decision_regions(X_decision, y_train, clf=Multiclf_D , legend=2)

# Adding axes annotations
plt.xlabel('sepal length [cm]')
plt.ylabel('petal length [cm]')
plt.title('Multiclf on Iris')
plt.show()


In [ ]:
#Draw decision boundaries
X_decision = X_train[:, [0, 2]]


# Training a classifier
Compclf_D = ComplementNB()
Compmodel_D = Compclf_D.fit(X_decision, y_train)
Compmodel_D

# Plotting decision regions
plot_decision_regions(X_decision, y_train, clf=Compclf_D , legend=2)

# Adding axes annotations
plt.xlabel('sepal length [cm]')
plt.ylabel('petal length [cm]')
plt.title('Compclf on Iris')
plt.show()


In [ ]:
#Draw decision boundaries
X_decision = X_train[:, [0, 2]]


# Training a classifier
Bernclf_D = BernoulliNB()
Bernmodel_D = Bernclf_D.fit(X_decision, y_train)
Bernmodel_D

# Plotting decision regions
plot_decision_regions(X_decision, y_train, clf=Bernclf_D  , legend=2)

# Adding axes annotations
plt.xlabel('sepal length [cm]')
plt.ylabel('petal length [cm]')
plt.title('Bernclf on Iris')
plt.show()


## 3-D

### All data

In [ ]:
from plotly.express import scatter_3d
# Plotting in 3D by plotly.express that would show the plot with capability of zooming,
# changing the orientation, and rotating
scatter_3d(iris, x='sepal length (cm)', y='sepal width (cm)', z='petal length (cm)', size="petal width (cm)",
                   color="species", color_discrete_map={"Joly": "blue", "Bergeron": "violet", "Coderre": "pink"})\
            .show()

### Model predictions

#### Gaussian

In [ ]:
scatter_3d(x=X_val[:, :1].flatten(), 
           y=X_val[:, 1:2].flatten(), 
           z=X_val[:, 2:3].flatten(), 
           size=X_val[:, 3:4].flatten(),
           color=GNB_pred.flatten(), 
           color_discrete_map={"Joly": "blue", "Bergeron": "violet", "Coderre": "pink"},
           labels={'x':'Sepal length (cm)', 'y':'Sepal width (cm)', 'z':'Petal length (cm)', 'size':'Petal width (cm)','color':'Species'})\
            .show()

### Multinomial

In [ ]:
scatter_3d(x=X_val[:, :1].flatten(), 
           y=X_val[:, 1:2].flatten(), 
           z=X_val[:, 2:3].flatten(), 
           size=X_val[:, 3:4].flatten(),
           color=Multi_pred.flatten(), 
           color_discrete_map={"Joly": "blue", "Bergeron": "violet", "Coderre": "pink"},
           labels={'x':'Sepal length (cm)', 'y':'Sepal width (cm)', 'z':'Petal length (cm)', 'size':'Petal width (cm)','color':'Species'})\
            .show()

#### Complement

In [ ]:
scatter_3d(x=X_val[:, :1].flatten(), 
           y=X_val[:, 1:2].flatten(), 
           z=X_val[:, 2:3].flatten(), 
           size=X_val[:, 3:4].flatten(),
           color=Comp_pred.flatten(), 
           color_discrete_map={"Joly": "blue", "Bergeron": "violet", "Coderre": "pink"},
           labels={'x':'Sepal length (cm)', 'y':'Sepal width (cm)', 'z':'Petal length (cm)', 'size':'Petal width (cm)','color':'Species'})\
            .show()

#### Bernoulli

In [ ]:
scatter_3d(x=X_val[:, :1].flatten(), 
           y=X_val[:, 1:2].flatten(), 
           z=X_val[:, 2:3].flatten(), 
           size=X_val[:, 3:4].flatten(),
           color=Bern_pred.flatten(), 
           color_discrete_map={"Joly": "blue", "Bergeron": "violet", "Coderre": "pink"},
           labels={'x':'Sepal length (cm)', 'y':'Sepal width (cm)', 'z':'Petal length (cm)', 'size':'Petal width (cm)','color':'Species'})\
            .show()

# 7. Models Comparison

In [ ]:
models = ['Gaussian', 'Multinomial', 'Complement', 'Bernoulli']
accuracies = [GNB_acc, Multi_acc, Comp_acc, Bern_acc]

for model, accuracy in zip(models, accuracies):
  print(model)
  print(round(accuracy,2))
  print('----------------')

In [ ]:
plt.figure(figsize=(8,6))
ax = sns.barplot(x = models, y = accuracies)

plt.xlabel('Models')
plt.ylabel('Accuracy')
plt.title('Comparing Accuracy of Models');

> Therefore, `Gaussian Naive Bayes` is the best model with `0.93 accuracy`

In [ ]:
# Model Prediction on Test
GNB_pred_test = GNBclf.predict(X_test)

# Model Accuracy
GNB_acc_test = calculate_accuracy(y_test, GNB_pred_test)
print('Gaussian Naive Bayes accuracy score: {0:0.4f}'. format(GNB_acc_test))
print(classification_report(y_test, GNB_pred_test))